In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
%matplotlib inline

In [2]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
label="Arched_Eyebrows"
image_dir="Data\img_align_celeba"

#define composed transforms
composed = transforms.Compose([transforms.ToTensor()])

ds=Binary_Dataset(labels_dir, label,image_dir ,transform=composed)
dataset_loader=DataLoader(ds,batch_size=32, shuffle=True, num_workers=2)

"""
for idx, (data, image) in enumerate(ds):
    print(idx)
"""

data=next(iter(dataset_loader))
sample, target = data
print(sample)

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Subin\anaconda3\envs\Pytorch_env\lib\site-packages\torch\utils\data\_utils\worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Subin\anaconda3\envs\Pytorch_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Subin\anaconda3\envs\Pytorch_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "d:\DeepLearning_Project\custom_dataset.py", line 39, in __getitem__
    image = io.imread(image_path)
  File "C:\Users\Subin\anaconda3\envs\Pytorch_env\lib\site-packages\skimage\io\_io.py", line 48, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "C:\Users\Subin\anaconda3\envs\Pytorch_env\lib\s

In [ ]:
plt.imshow(torch.squeeze(data[0]))
print(target[0])